In [3]:
import sqlite3
import json
from typing import List, Dict, Optional

def get_sqlite_schema(db_path: str) -> Dict:
    """Generate schema JSON from SQLite database."""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Get all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    schema = {
        "name": "YOUR_SCHEMA_NAME",  # Replace with your schema name
        "description": "Schema generated from SQLite database",
        "tables": [],
        "foreign_keys": []
    }
    
    for table_name in tables:
        table_name = table_name[0]
        if table_name == 'sqlite_sequence':  # Skip SQLite internal tables
            continue
            
        # Get table info
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns_info = cursor.fetchall()
        
        # Get foreign keys
        cursor.execute(f"PRAGMA foreign_key_list({table_name});")
        foreign_keys = cursor.fetchall()
        
        table = {
            "name": table_name,
            "description": f"Table {table_name}",
            "columns": []
        }
        
        # Process columns
        for col in columns_info:
            column = {
                "name": col[1],  # column name
                "type": col[2],  # column type
                "description": f"Column {col[1]} of type {col[2]}",
                "example": None  # You might want to add example values if available
            }
            table["columns"].append(column)
        
        schema["tables"].append(table)
        
        # Process foreign keys
        for fk in foreign_keys:
            fk_constraint = f"{table_name}.{fk[3]} -> {fk[2]}.{fk[4]}"
            schema["foreign_keys"].append(fk_constraint)
    
    conn.close()
    return schema

def save_schema_to_json(schema: Dict, output_path: str):
    """Save schema to JSON file."""
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(schema, f, indent=2)

# Usage example:
db_path = "E:/code/AI/agentic-AI/SQL-QA/Chinook.db"  # Replace with your SQLite database path
output_path = "E:/code/AI/agentic-AI/SQL-QA/data/GSV/M-Schema/chinbook.json"   # Replace with desired output path

schema = get_sqlite_schema(db_path)
save_schema_to_json(schema, output_path)